--------測試retriever----------

In [1]:
from pyserini.search.lucene import LuceneSearcher
from pyserini.index import IndexReader

def Retreiver(quesion):
  searcher.set_language('zh')
  hits = searcher.search(quesion, k=10)
  for i in range(len(hits)):
    print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f}')

  return hits
searcher = LuceneSearcher('./indexes/')
index_reader = IndexReader('./indexes')


/user_data/miniconda3/envs/model_training/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#改變wiki資料格式，並額外創出做出反向索引需要的檔案

import pandas as pd
import os, json

folder_path = "Training Dataset_v2/訓練資料集/wiki-pages_Dec 2022 dump/wiki-pages"
files = os.listdir(folder_path)
data_total=[]
for i in range(len(files)):
    data=[]
    with open(f'Training Dataset_v2/訓練資料集/wiki-pages_Dec 2022 dump/wiki-pages/{files[i]}', 'r', encoding='utf-8') as f:
        data_read = f.read()  # Read the contents of the file into a string
        data_read = data_read.strip().split('\n')  # Split the string into a list of lines
        for line in data_read:
            line = json.loads(line)
            data_total.append(line)
            data.append(line)
data_total = pd.DataFrame(data_total)

In [3]:
#處理public_train得到的claim，並retrieve回文章

from udicOpenData.dictionary import *
from udicOpenData.stopwords import *

pd.set_option("display.max_colwidth", 100000) #要加這行，不然底下dataframe的資訊只會出現一半
#找出answer_id_line_list
public_train = []
with open(f'Training Dataset_v2/訓練資料集/public_train_0316.jsonl', 'r', encoding='utf-8') as f:
    data_read = f.read()  # Read the contents of the file into a string
    data_read = data_read.strip().split('\n')  # Split the string into a list of lines
    for line in data_read:
        line = json.loads(line)
        public_train.append(line)

answer_id_line_list = []
claim =public_train[53]['claim']
for second in public_train[53]['evidence']:
    for third in second:
        answer_id_line_list.append([third[2],third[3]])
print(answer_id_line_list)

#找出retrieved_id
hints=""
for i in rmsw(claim,flag=True):
    print(i)
    if (i[1] not in ['v','ng']) and (i[0] not in ["原因", "為何"]):
        hints = hints+" "+i[0]
retrieved_data = Retreiver(claim)
retrieved_id=[]
for i in retrieved_data:
    id = json.loads(i.raw)["id"]
    if id not in retrieved_id:
      retrieved_id.append(id)
print(retrieved_id)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.521 seconds.
Prefix dict has been built successfully.
[nltk_data] Downloading package punkt to /user_data/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /user_data/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /user_data/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /user_data/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /user_data/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


[['竇唯', 7], ['竇唯', 0], ['竇唯', 6]]
('離開', 'v')
('黑豹樂隊', 'n')
('後', 'n')
('著名', 'a')
('中國音樂', 'n')
('人', 'n')
('竇唯', 'n')
('音樂風格', 'n')
('擺脫', 'n')
('過往', 't')
('流行搖滾', 'n')
 1 竇唯   57.50560
 2 竇唯音樂作品列表 51.87510
 3 幻聽_(專輯) 46.89600
 4 黑豹樂隊 40.16100
 5 巒樹   37.51560
 6 豔陽天_(專輯) 35.92970
 7 山河水  35.68700
 8 美國音樂 33.98910
 9 八段錦_(專輯) 33.88930
10 新浪潮音樂 31.80090
['竇唯', '竇唯音樂作品列表', '幻聽_(專輯)', '黑豹樂隊', '巒樹', '豔陽天_(專輯)', '山河水', '美國音樂', '八段錦_(專輯)', '新浪潮音樂']


In [8]:
public_train[53]

{'id': 2235,
 'label': 'supports',
 'claim': '離開黑豹樂隊後，著名中國音樂人竇唯的音樂風格擺脫過往的流行搖滾。',
 'evidence': [[[3762, 3914, '竇唯', 7],
   [3762, 3914, '竇唯', 0],
   [3762, 3914, '竇唯', 6]]]}

In [11]:
data_total.loc[data_total['id'] == '竇唯', 'lines'].to_string(index=False).split('\\n')

['0\\t竇唯  ， 中國大陸著名音樂家 、 歌手 、 作曲家 、 音樂製作人 ， 中國搖滾樂代表人物之一 。\\t中國大陸\\t中國大陸\\t音樂家\\t音樂家\\t歌手\\t歌手\\t作曲家\\t作曲家\\t音樂製作人\\t音樂製作人\\t中國搖滾樂\\t中國搖滾樂',
 '1\\t',
 '2\\t',
 '3\\t1994年5月發行第一張個人專輯 《 黑夢 》 ， ， 與魔巖唱片同儕張楚 、 何勇並稱爲 “ 魔巖三傑 ” 。\\t黑夢\\t黑夢 (專輯)\\t張楚\\t張楚 (歌手)\\t何勇\\t何勇 (歌手)\\t魔巖三傑\\t魔巖三傑',
 '4\\t',
 '5\\t',
 '6\\t1988年 ， 竇唯接替丁武擔任黑豹樂隊主唱 ， 並於1991年出版了專輯 《 黑豹 》 ， 迅速在香港和大陸取得了商業上的成功 ， 這是新中國第一張流行搖滾樂專輯 。\\t流行搖滾\\t流行搖滾\\t黑豹樂隊\\t黑豹樂隊',
 '7\\t之後竇唯離開黑豹 ， 組建了做夢樂隊 （ 1991年 ， 1992年解散 ） 、 譯樂隊 （ 1999年 ） 、 不一定樂隊 、 暮良文王 （ 2002年 ） 和不一樣樂隊 ， 其音樂風格開始多元化 。\\t黑豹\\t黑豹 (專輯)\\t做夢樂隊\\t做夢樂隊\\t譯樂隊\\t譯樂隊\\t不一定樂隊\\t不一定樂隊\\t暮良文王\\t暮良文王\\t不一樣樂隊\\t不一樣樂隊',
 '8\\t近年來竇唯多次和朝簡樂隊合作 。\\t朝簡樂隊\\t朝簡樂隊',
 '9\\t',
 '10\\t',
 '11\\t竇唯爲王菲的前夫 。\\t王菲\\t王菲',
 '12\\t']

In [12]:
import pandas as pd
pd.read_csv('/user_data/AI_CUP_2023_事實文字檢索與查核/Training Dataset_v2/訓練資料集/sentence_retrieval_train.csv')


,claim,narrative,labels
0,天衛三軌道在天王星內部的磁層，以《 仲夏夜之夢 》作者緹坦妮雅命名。,1787年由威廉 · 赫歇爾發現 ， 並以威廉 · 莎士比亞的 《 仲夏夜之夢 》 中的妖精王后緹坦妮雅命名 。仲夏夜之夢仲夏夜之夢緹坦妮雅緹坦妮雅,evidence
1,信天翁科的活動範圍位於北冰洋以及南太平洋，牠的翼展可達到3.7米，是世界上現存的翼展最大的鳥類。,北大西洋不在它們的活動範圍 ， 但當地的化石記錄顯示它們過去曾生活於此 ， 另外偶爾也會有迷鳥到達這裏 。大西洋大西洋迷鳥迷鳥,non-evidence
2,南京大學附屬中學，從中國江蘇省遷移。,南開大學附屬中學 ， 位於中國天津市南開區 。中國中國南開大學附屬中學南開大學附屬中學天津市天津市南開區南開區,non-evidence
3,南京大學附屬中學，從中國江蘇省遷移。,南京大學附屬中學 ， 位於中國江蘇省南京市 。南京大學附屬中學南京大學附屬中學中國中國江蘇省江蘇省南京市南京市,evidence
4,毒魚豆的萃取物被西印度群島的原住民發掘可以導致魚麻醉安靜 ， 讓他們得以趁機徒手抓魚 。,原產於美國佛羅里達州南部 、 佛羅里達礁島群 、 德克薩斯州 ， 加勒比地區及拉丁美洲 。佛羅里達礁島群佛羅里達礁島群德克薩斯州德克薩斯州,non-evidence
...,...,...,...
7696,備後國的福山城現今位於日本廣島縣，曾在戰爭中遭到攻擊。,日本國家指定史蹟 ， 久松城 （ ひさまつじょう ） 、 葦陽城 （ いようじょう ） 是此城別名 。,non-evidence
7697,備後國的福山城現今位於日本廣島縣，曾在戰爭中遭到攻擊。,Category : 廣島縣城堡,non-evidence
7698,備後國的福山城現今位於日本廣島縣，曾在戰爭中遭到攻擊。,在第二次世界大戰期間 ， 此國寶天守遭到美軍攻擊而燒燬 。,evidence
7699,世界上僅次葡萄酒的品牌爲蘋果酒。,蘋果酒相對比其它水果酒 ， 其水果的氣味濃烈 、 甜度高 ， 對於不喜歡喝酒精飲料的人來說適口性非常強 。水果酒水果酒蘋果蘋果,non-evidence


In [5]:
import re, random

def add_context(answer_id_line_list, answer_context, id_line, line, i):
    if not re.match(r'^\d+\\t$',line): #不能是"[數字]\\t"的句子
        #不能是答案裡的句子
        for id_line_compare in answer_id_line_list:
            if id_line[0] == id_line_compare[0] and i == id_line_compare[1] : 
                return False
        context = re.sub(r'^\d+', '', line)
        context = context.replace("\\t",'')
        if context in answer_context:
            return True
        answer_context.append(context)
        return True
    return False

answer_context = [] #答案的上下文
for id_line in answer_id_line_list:
    line_list = data_total.loc[data_total['id'] == id_line[0], 'lines'].to_string(index=False).split('\\n')
    #找上文
    for i in range(id_line[1]-1,-1,-1):
        if add_context(answer_id_line_list, answer_context, id_line, line_list[i], i):
            break
    #找下文
    for i in range(id_line[1]+1,len(line_list)):
        if add_context(answer_id_line_list, answer_context, id_line, line_list[i], i):
            break
answer_context = random.sample(answer_context, len(answer_context))

In [6]:
import math

#for public_train_line in public_train:
positive_data = []
for line in answer_id_line_list:
    evidence = data_total.loc[data_total['id'] == line[0], 'lines'].to_string(index=False).split('\\n')[line[1]]
    evidence = re.sub(r'^\d+', '', evidence)
    evidence = evidence.replace("\\t",'')
    positive_data.append(evidence)

negative_data=[]
n = len(answer_id_line_list)
match_n_limit = math.ceil((3.0/5)*n)

answer_context = answer_context[:match_n_limit]
negative_data.extend(answer_context)
n_out = n - len(negative_data)
out_retrieved_id = retrieved_id.copy()


In [7]:
out_retrieved_id = random.sample(out_retrieved_id, n_out)
out_retrieved_negative_data = []
for i in range(n_out):
    line_list = data_total.loc[data_total['id'] == out_retrieved_id[i], 'lines'].to_string(index=False).split('\\n')
    choose = random.choice(line_list)
    while re.match(r'^\d+\\t$',choose): 
        line_list.remove(choose)
        choose = random.choice(line_list)    
    choose = re.sub(r'^\d+', '', choose)
    choose = choose.replace("\\t",'')
    out_retrieved_negative_data.append(choose)
negative_data.extend(out_retrieved_negative_data)
assert len(negative_data) == n